### week6 크롤링 assignment - 최혜빈

# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [20]:
#필요한 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup
import re

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [21]:
#영화제목은 뒤에서 함께 출력할 때 자주 사용할 것 같아서 따로 함수로 만들었습니다
def movie_title():
    url = 'https://movie.naver.com/movie/running/current.nhn' #url 설정
    res = requests.get(url) #url에 request
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    data = soup.find('ul', class_='lst_detail_t1') #영화제목만 쭉 뽑아내기 위해 우선 이 클래스를 찾았고!
    data_a = []
    for tag in data.find_all('a'): #그 중 a태그부분만 가져왔습니다 
        data_a.append(tag.text) #이부분은 영화제목, 장르, 감독, 배우가 모두 포함되어 있는데 영화마다 ''공백으로 나누어져있어서 이를 이용해 제목만 뽑아냈습니다
        
    movie = []
    for i in range(len(data_a)):
        if data_a[i] == '': #앞서 가져온 데이터를 돌면서 ''공백을 확인하면 그 다음 인덱스에 존재하는 영화제목을 뽑아내도록 하였습니다
            movie.append(data_a[i+1])
        if len(movie) == 5: #영화제목 5개만 뽑아내기 위해 5개가 되면 break!
            break
    return movie #이 함수를 실행하면 영화제목 5개을 return값으로 받아옴
    

In [22]:
#예매순 상위 5개의 현재 상영 중 영화 제목과 주연배우 3인 가져오는 함수!
def movie_title_url_actor():
    url = 'https://movie.naver.com/movie/running/current.nhn' #url 설정
    res = requests.get(url) #url에 request
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    movie = movie_title() #위에서 정의한 함수로 영화제목 5개를 받아옵니다

    actor_all= []
    for i in range(len(movie)):
        data = soup.select('span[class="link_txt"]')[2+3*i] #link_txt class를 확인하면 각 영화의 개요,감독,출연이 순서대로 반복되어서 나옵니다
        #출연 배우를 뽑아내기때문에 index 2, 5, 8....을 뽑아내도록 설정했습니다
        actor = []
        for tag in data.select('a'): #출연에 대한 data에서 a태그만 뽑아내고, text만 뽑아내게 하여서 주연 배우 3명을 뽑아냅니다
            actor.append( tag.text) #이 actor list는 한 영화에 대한 영화배우 list
            if len(actor) == 3:
                break
        actor_all.append(actor) #이 actor_all list는 5개 영화에 각각 주연배우 3명씩 actor list가 들어있는 list
    
    movie_dict = {}
    #딕셔너리로 만들어준다
    #5개의 영화제목이랑 영화배우들을 묶어줌!
    for i in range(len(movie)): 
        movie_dict[movie[i]] = actor_all[i]
    
    print(movie_dict) # 이 함수를 실행하면 영화제목과 주연배우 3인이 출력되도록 함.
    
    return movie, actor_all #마지막에 데이터프레임으로 합쳐주기 위해서, save함수의 인자값으로 사용하기 위해 return 값 설정


### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [23]:
def get_grade():
    url = 'https://movie.naver.com/movie/running/current.nhn' #url 설정
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    movies = soup.select("dl.lst_dsc") 
    #해당 영화의 세 평점을 모두 가져오기 위해서는 해당 영화의 url을 들어가야 하므로 해당 url들을 영화마다 구해 list로 만들었습니다 
    url_all = []
    for m in movies :
        title = m.select_one("dt.tit a") 
        url = title.attrs['href'] #dt.tit의 a태그 중 attr['href']부분들만 뽑아오면 해당페이지로 들어가는 url부분이 나오더라고요 
        url_all.append("https://movie.naver.com" + url) #이를 이용해서 url 5개를 완성했습니다
        if len(url_all) == 5:
            break
            
    movie = movie_title() #앞서 정의해둔 함수 이용해서 영화 5편의 제목 가져오기
            
            
    score_all = []
    for i in url_all: #앞서 구한 url마다 각각 평점을 뽑아온다
        url = i
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
    
        score = soup.select('div[class="star_score"] em', limit=12) #평점을 가지고있는 태그를 가져옴
        score_1 = []
        add = ''
        for i in range(len(score)):   
            add += score[i].get_text()
            if i in list(range(3,12,4)): #'9','.','0','7'이런식으로 되어있어서 각각 묶어서 list로 저장하기 위해 for문 이용
                score_1.append(add) #score_1은 영화 한편당 해당 평점들 저장 list
                add = ''
        score_all.append(score_1) #score_all은 5편의 영화 각각 평점 모두 저장한 list
        
    score_dict = {}
    
    #영화제목과 평점들을 함께 출력하기 위해 딕셔너리로 묶어줌
    for i in range(len(movie)): 
        score_dict[movie[i]] = score_all[i]

    print(score_dict) # 이 함수 실행시 딕셔너리 형태로 영화제목과 평점들 주르륵 나오도록!
    
    return score_all #이건 맨 마지막에 데이터프레임 저장할 때 사용위해 return 값으로 받는다

### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

In [24]:
#아래함수 안에 넣기에는 너무 길어져서 따로 함수로 정했습니다
def function_review(url_all):    # 이함수는 아래서 구하는 url에 들어갔을 때 그 해당 url에서 평점, 평점 작성자 id, 리뷰 본문을 받아옵니다
        review_result = []
        for i in url_all: # 함수인자로 들어오는 url은 5개의 영화에 대한 url들이 들어있는 list형태이므로 for문으로!
            url = i #url설정
            res = requests.get(url)
            html = res.text
            soup = BeautifulSoup(html, 'html.parser')

            review = soup.select('div[class="score_reple"] span' ) # score_reple class의 span 태그를 확인하면 리뷰본문과 id가 번갈아가면서 나옵니다


            review_id = []

            #먼저 id를 가져오기 위해 앞서 받은 데이터(리뷰본문과 id)에서 닉네임(id****)형태로 되어있는 문자열들만 가져오도록해서 id를 뽑아옵니다
            id_check1 = re.compile('\w*\s*\w*\(*\w*[*]{4}\)*') #정규표현식 사용!
            id_check2 = re.compile('\w*\(*\w*[*]{4}\)*') 

            for i in range(len(review)):
                check1 = id_check1.match(review[i].get_text().strip())
                check2 = id_check2.match(review[i].get_text().strip())

                if check1 or check2: #위에서 설정한 형태에 알맞으면 id라고 인식해서 id 리스트에 들어가도록 함
                    review_id.append(review[i].get_text().strip())


            review_text = []
            #다음은 리뷰 본문을 가져옵니다
            
            for i in range(len(review)):
                text = review[i].get_text().strip() # 위에서 구한 데이터를 그대로 사용하되
                if (text in review_id) or (text == '관람객'): #위에서 이미 id라고 설정한 문자열과 중간중간에 관람객이라고 설정되어있는 리뷰본문에 필요없다고 판단한 문자열을 제외한 리뷰본문만 
                    pass
                else:
                    review_text.append(text) #review_text에 들어갑니다
            
            #다음은 평점들을 가져옵니다
            score = soup.select('div[class="star_score"] em' ) #평점을 가져오는 다른 class의 em 태그를 사용
            review_score = []
            for i in range(len(score)):
                review_score.append(score[i].get_text()) #평점들을 쭉 가져와서 review_score list에 저장한다

            review_page = []

            for i in range(len(review_score)): #앞서 구한 평점/ 리뷰본문/ 닉네임(id) 를 각각 묶어서 새로운 리스트에 저장
                review_page.append(review_score[i]+ ' / ' +review_text[i]+ ' / '+review_id[i]) #review_page는 한 영화에 대한 리뷰들을 쭉
            review_result.append(review_page) #review_result는 5편의 영화에 대한 리뷰들 list를 모두 포함하는 list!

        return review_result # 구한 리스트를 return값으로 가져옵니다

In [25]:
def get_reviews():
    url = 'https://movie.naver.com/movie/running/current.nhn' #url 설정
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    movies = soup.select("dl.lst_dsc")

    movie = movie_title() #앞서 정의한 함수 이용해서 영화 제목들 받아온다

    url_all_1 = []
    url_all_2 = []
    for m in movies :
        title = m.select_one("dt.tit a") #위에서 평점구할때 사용했던 방법들과 비슷함
        url = title.attrs['href']
        code = url[-6:] #각각 영화에 따라 달라지는 페이지의 code번호들을 받아온다
        #리뷰를 받아오는 url은 모바일버전을 사용했습니다!
        url_all_1.append("https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=" + code+'&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1')
        #한 페이지당 10개씩 있어서 페이지 1일때, 2일때 따로 url을 저장했습니다
        url_all_2.append("https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=" + code+'&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2')
        #앞서 구한 각 영화당 코드번호들을 이용해서 평점을 구하는데 필요한 모든 url을 list에 저장!
        if len(url_all_1) == 5:
            break
            
    #위에서 정의한 함수를 통해 page1의 url일때 각영화의 리뷰들 10개씩, 똑같이 page2에서 10개씩 받아옵니다
    review_page_1 = function_review(url_all_1)
    review_page_2 = function_review(url_all_2)

    review_page = []
    #page1과 page2을 더해줍니다
    for i in range(len(review_page_1)):
        review_page.append(review_page_1[i]+review_page_2[i])


    review_dict = {}
    #영화별로 dictionary로 만들어서 제목과 평점/닉네임/본문이 쭉 나오도록 했습니다

    for i in range(len(movie)): 
        review_dict[movie[i]] = review_page[i]

    print(review_dict) # 이 함수를 실행했을 때 review에 대해서 쭉 출력되게 함

    return review_page #이건 나중에 dataframe으로 저장할때 필요하니까 return값으로 !

### 4. 저장하기

In [26]:
#앞서 구한 정보들을 인자로 받아와서 모두 합쳐서 dataframe형태로 만들고 csv, excel형태로 저장하는 함수
def save(movie,actor,score,review): #앞서 구한 데이터들을 인자로 받아옵니다
    import pandas as pd
     
    df = pd.DataFrame() #dataframe형태로 만들었습니당
    df['영화'] = movie
    df['주연배우'] = actor
    df['네티즌 평점, 관람객 평점, 기자/평론가 평점'] = score
    df['평점/평점 작성자 닉네임/리뷰 본문'] = review
    
    #각 column들에 데이터들을 각각 넣어주고
    #csv형태와 xlsx로 형태로 저장시켜준다!
    df.to_csv('naver_movie.csv')
    df.to_excel('naver_movie.xlsx')

### 5. 크롤링하기

In [27]:
data = movie_title_url_actor() #예매순 상위 5개의 현재 상영 중인 영화 가져오는 함수(출력!)

{'지푸라기라도 잡고 싶은 짐승들': ['전도연', '정우성', '배성우'], '정직한 후보': ['라미란', '김무열', '나문희'], '1917': ['조지 맥케이', '딘-찰스 채프먼'], '작은 아씨들': ['시얼샤 로넌', '엠마 왓슨', '플로렌스 퓨'], '클로젯': ['하정우', '김남길', '허율']}


In [28]:
movie = data[0] #영화제목들 
actor = data[1] #주연배우들

In [29]:
data = get_grade() #해당 영화의 평점 가져오는 함수(네티즌 평점, 관람객 평점, 기자/평론가 평점) (출력!)

{'지푸라기라도 잡고 싶은 짐승들': ['8.67', '6.71', '7.18'], '정직한 후보': ['8.71', '5.38', '7.73'], '1917': ['9.40', '7.67', '9.02'], '작은 아씨들': ['9.24', '8.00', '8.89'], '클로젯': ['8.41', '5.50', '6.87']}


In [30]:
score = data #평점들(네티즌 평점, 관람객 평점, 기자/평론가 평점)

In [31]:
data = get_reviews() #관람객 평점 공감순 20건 가져오는 함수(평점, 평점 작성자 닉네임, 리뷰 본문)

{'지푸라기라도 잡고 싶은 짐승들': ['10 / 난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 지푸라기 / bohemian(mabu****)', '10 / 전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임... / 최정규(cjg4****)', '10 / 8명의 배우가 모두 주인공 같은 느낌. / 달다(fxko****)', '1 / 아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽부터 함? 백번양보해서 시사회때 봤다 쳐도 이렇게나 많이 봤다고? 죄다 똑같은 말투에? 음원이고 영화고 조작질 역겹다 진짜 / 어쩌라고(dpfk****)', '9 / 개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ / 써니(tlag****)', '10 / 연출, 연기, 스토리 모두 대박...무조건 보세요. / 까칠소녀(oper****)', '9 / 진짜 보고싶었던 영환데 드디어 봤습니당 기다린 보람이 있네용ㅋㅋㅋ 등장인물이 많았는데 영화 속에서 잘 풀어낸 것 같아요 강추합니당 !! / haeunnnnn(0_80****)', '9 / 한국식 피칠갑을 한 타란티노 영화 / hojo****', '9 / 연기오지고 스릴오지고 / Cjin(dlck****)', '10 / 스토리가 짱짱하네요~ 심리적인 긴장감을 잘 살린 영화인것 같네요~ 인기좀 끌듯... / trau****', '10 / 연기 쩐다잉 ,,, 또 보고 싶음 ㅠ / 꽁끼(bamb****)', '1 / 방금 보고 왔는데 지금 심정이 지푸라기라도 잡고 싶은 심정이다 / osk1****', '10 / 전도연을 위한, 전도연에 의한 영화! 데뷔작이라고는 믿을수 없는 연출력에놀랐다~ / Linus(getu****)', '10 / 이 캐스팅 너무 마음에 든다.영화보고나서도 할말이 많아지는 영화 / myd5q3ji7(i2g1****)', '9 / 솔직히 이 영화 돈주고 볼만합니다ㅎㅎ / 파리투나잇(lgxe****)', '1 / 영화 보는 내내 

In [32]:
review = data #review

In [33]:
save(movie,actor,score,review) #파일로 저장!